In [10]:
import pydicom as pydcm
import matplotlib.pyplot as plt
import pandas as pd
import sys
from pydicom.datadict import dictionary_VR
from pydicom import dcmread,Dataset
from pydicom.filewriter import dcmwrite
import numpy as np
#import cv2
import random
import time
import datetime


from google.cloud import storage
from io import BytesIO
client = storage.Client()

In [11]:
def blob_to_image(bucket, blob_name):
    blob = bucket.get_blob(blob_name)
    blobstring = blob.download_as_string()
    dicomfile = BytesIO(blobstring)
    dicomread = dcmread(dicomfile)
    #plt.imshow(dicomread.pixel_array)
    return dicomread

def img_diff(raw, deid):
    diff = raw.pixel_array-deid.pixel_array
    result = np.count_nonzero(diff)
    return(result)    

def find(list,thing): 
    "finds indices of something in a list"
    return [i for i, j in enumerate(list) if j == thing]


def blob_to_df(filelist, bucket):
    SOP = []
    patient = []
    study = []
    series = []
    DICOM = []
    for i in range(len(filelist)):
        dicom = blob_to_image(bucket, filelist[i])
        DICOM.append(dicom)
        patient.append(dicom.PatientID)
        study.append(dicom.StudyInstanceUID)
        series.append(dicom.SeriesInstanceUID)
        SOP.append(dicom.SOPInstanceUID)
    
    d = {'dicom': DICOM, 'patientID': patient, 'studyID': study, 'seriesID': series, 
         'SOPID': SOP}
    df = pd.DataFrame(d)
    return(df)


In [16]:
bucket_in = client.get_bucket('midi-import')

blobs_in = bucket_in.list_blobs(prefix='ds1_1-600/group2/3470831496')
filelist_in = [item.name for item in blobs_in] #extracting file names from blobs

df = blob_to_df(filelist_in[1:], bucket_in)
#df_in = blob_to_df(filelist_in[1:], bucket_in)

/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.4.637.1.1.0286981.2.191.3459773422647729824'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.4.637.1.1.0286981.2.191.1893866593186486911'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.4.637.1.1.0286981.2.191.2611140455051373536'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.4

In [24]:
#automatic code final

file = 0
vrs = ['LO', 'SH', 'LT', 'ST', 'UT','IS']
str1 = ''
x = []
x2 = []

patient = []
study = []
SOP = []
series = []
element = []
tag = []
value = []
vr = []
newvalue = []

#loops through each dicom file imported
for i in range(len(df)):
    file = df.iloc[i][0]
    #loops through each element in the dicom file
    for elem in file.iterall():
        #only looks at elements of vr 'LO', 'SH', 'LT', 'ST', 'UT','IS'
        #Also includes (incorrect) attempt to ignore Code Value elements
        if elem.VR in vrs and elem.value != 'CodeValue':
            #ignores elements that don't contain any numeric characters
            if str(elem.value).isalpha() == False:
                str1 = str(elem.value)
                try:
                    #creates list of all continuous numeric strings of characters in the element
                    x = re.findall('[0-9]+', str1)
                    #narrows this list to only strings of length 6 or longer
                    x2 = list(filter(filterLength, x))
                    if len(x2) != 0:
                        #appends the element info to lists. This is not necessary for de-id, but is used to double check the results
                        patient.append(df.iloc[i][1])
                        study.append(df.iloc[i][2])
                        series.append(df.iloc[i][3])
                        SOP.append(df.iloc[i][4])
                        element.append(str(elem))
                        tag.append(elem.name)
                        vr.append(elem.VR)
                        value.append(str(elem.value))
                        #loops through the list of continuous numeric strings in the element
                        for n in x2:
                            if len(n) == 8:
                                #if the string's length is 8, check if it is a valid yyyymmdd date
                                yr = n[:4]
                                m = n[4:6]
                                d = n[6:]
                                if 1895<=int(yr)<=2023 and 0<int(m)<=12 and 0<int(d)<=31 and elem.VR != 'IS':
                                    #if it is a valid yyyymmdd date and isn't in VR IS, reformat it to yyyy-mm-dd
                                    str3 = yr+'-'+m+'-'+d
                                    str4 = str1.replace(n,str3)
                                    elem.value = str4
                                else:
                                    #replaces the 8 length int string with nothing
                                    str2 = str1.replace(n,'')
                                    elem.value = str2
                            else:
                                #replaces the int string (not 8 length) with nothing
                                str2 = str1.replace(n,'')
                                elem.value = str2
                        newvalue.append(str(elem.value))
                                
                except:
                    print(elem)
                                      
                        
    

In [18]:
df[df['SOPID']=='2.1.161.0.1.1581247.1.844.1680903824385914599']

,dicom,patientID,studyID,seriesID,SOPID
99,"[(0008, 0005) Specific Character Set ...",3470831496,2.1.161.0.1.1581247.1.844.1645250445130037887,2.1.161.0.1.1581247.1.844.1197658322026313357,2.1.161.0.1.1581247.1.844.1680903824385914599


In [25]:
df.iloc[99][0].AdditionalPatientHistory

'Ordered for 675-36-2449 by Dr. Davis'

In [127]:
d = {'patient':patient, 'study':study, 'series':series, 'SOP':SOP, 'tag':tag, 'vr':vr, 'element':element, 'value':value, 'newvalue':newvalue}
flagged = pd.DataFrame(d)

In [128]:
flagged.groupby(['tag'])['tag'].count()

tag
Accession Number                        353
Acquisition Comments                    245
Additional Patient History              498
Attenuation Correction Method           118
Clinical Trial Subject ID               263
Image Comments                           12
Imaging Service Request Comments          1
Medical Alerts                           95
Patient Comments                        495
Patient ID                              721
Private tag data                        200
Protocol Name                           190
Requested Procedure Comments            235
Requested Procedure Description         374
Scheduled Procedure Step Description     12
Series Description                      295
Series Number                           590
Study Comments                          174
Study Description                       326
Study ID                                407
Text Value                                2
Name: tag, dtype: int64

In [129]:
flagged[flagged['tag']=='Accession Number']

,patient,study,series,SOP,tag,vr,element,value,newvalue
421,1326414678,2.2.379.0.1.5971236.3.509.2117853776506399936,2.2.379.0.1.5971236.3.509.6412913489296797476,2.2.379.0.1.5971236.3.509.1118646326684973163,Accession Number,SH,"(0008, 0050) Accession Number ...",20170304E698535,-2017-03-04-E
425,1507439060,2.2.874.0.0.8400978.5.486.2510732133849908984,2.2.874.0.0.8400978.5.486.1747132007124743752,2.2.874.0.0.8400978.5.486.2019300706932341803,Accession Number,SH,"(0008, 0050) Accession Number ...",20170726E609517,-2017-07-26-E
505,1814567196,1.5.566.0.3.5921028.1.523.6121491292105488144,1.5.566.0.3.5921028.1.523.5821238071079767946,1.5.566.0.3.5921028.1.523.6321575997177780656,Accession Number,SH,"(0008, 0050) Accession Number ...",20180805E673674,-2018-08-05-E
988,4953814140,1.3.241.0.0.3328322.7.935.2685937606730192613,1.3.241.0.0.3328322.7.935.9222594282780660977,1.3.241.0.0.3328322.7.935.1641820698690379306,Accession Number,SH,"(0008, 0050) Accession Number ...",20171114E506367,-2017-11-14-E
994,4953814140,1.3.241.0.0.3328322.7.935.2685937606730192613,1.3.241.0.0.3328322.7.935.9222594282780660977,1.3.241.0.0.3328322.7.935.3386585305478125613,Accession Number,SH,"(0008, 0050) Accession Number ...",20171114E506367,-2017-11-14-E
...,...,...,...,...,...,...,...,...,...
5561,9473050187,2.5.444.1.3.9432617.9.170.1605644608138802224,2.5.444.1.3.9432617.9.170.1185165576631833947,2.5.444.1.3.9432617.9.170.1220587225385277156,Accession Number,SH,"(0008, 0050) Accession Number ...",20140722E804773,-2014-07-22-E
5570,9473050187,2.5.444.1.3.9432617.9.170.1605644608138802224,2.5.444.1.3.9432617.9.170.1185165576631833947,2.5.444.1.3.9432617.9.170.3053293679157621944,Accession Number,SH,"(0008, 0050) Accession Number ...",20140722E804773,-2014-07-22-E
5579,9473050187,2.5.444.1.3.9432617.9.170.1605644608138802224,2.5.444.1.3.9432617.9.170.1185165576631833947,2.5.444.1.3.9432617.9.170.2941125927500826196,Accession Number,SH,"(0008, 0050) Accession Number ...",20140722E804773,-2014-07-22-E
5588,9473050187,2.5.444.1.3.9432617.9.170.1605644608138802224,2.5.444.1.3.9432617.9.170.1185165576631833947,2.5.444.1.3.9432617.9.170.3114818742644719069,Accession Number,SH,"(0008, 0050) Accession Number ...",20140722E804773,-2014-07-22-E


In [130]:
df.iloc[0][0]['PatientID']

(0010, 0020) Patient ID                          LO: ''

In [136]:
flagged['newvalue'][90:300]

90                 Ordered for  *
91     PET-CT SERIES -2010-12-03-
92                               
93                 Ordered for  *
94                               
                  ...            
295    PET-CT SERIES -2010-12-03-
296                              
297                Ordered for  *
298                              
299                              
Name: newvalue, Length: 210, dtype: object

In [137]:
len(filelist_in)

722

In [9]:
bucket = client.get_bucket('midi-import')
for i in range(1,len(filelist_in)):
    filepath = 'preprocessed/ds1_1-23preprocessed6-27/'+filelist_in[i]
    blob = bucket.blob(filepath)
    df.iloc[i-1][0].save_as('test6.dcm')
    blob.upload_from_filename('test6.dcm')
    

In [7]:
str1 = 'Performed 20171126'

n = '20171126'

out1 = re.split('(19[0-9][0-9]|20[0-2][0-9])(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])',n)
dt1 = ' '.join(out1)

out2 = re.split('(0[1-9]|1[0-9]|2[0-9]|3[0-1])(0[1-9]|1[0-2])(19[0-9][0-9]|20[0-2][0-9])',dt1)
dt2 = ' '.join(out2)

out3 = re.split('(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])(19[0-9][0-9]|20[0-2][0-9])',dt2)
dt3 = ' '.join(out3)

dt3

if len(out1) == 1 and len(out2)==1 and len(out3)==1:
    str2 = str1.replace(n,'')
    
else:
    str2 = str1.replace(n,dt3)

print(dt1)
#this might end up being more computational expensive than the if statements

 2017 11 26 


In [103]:
df.iloc[100][0]['SeriesDescription'].value = 'PET-CT SERIES -2010-12-03-'

In [35]:
#automatic code final
import re

file = 0
#vrs to look at
vrs = ['LO', 'SH', 'LT', 'ST', 'UT','IS']
str1 = ''
x = []
x2 = []

patient = []
study = []
SOP = []
series = []
element = []
tag = []
value = []
vr = []
newvalue = []

#regex rules
loc = re.compile('at [A-Z]{2,4}')
dr = re.compile('by [A-Z]{2}')

#blacklist
blacklist = [' MGH', ' ALH']

for i in range(len(df)):
    file = df.iloc[i][0]
    for elem in file.iterall():
        if elem.VR in vrs and elem.name != 'Code Value' and elem.name != 'Private Creator':
            if elem.tag not in keeplist:
                str1 = str(elem.value)
                try:
                    #regex replacement
                    strloc = loc.sub('*',str1)
                    str1 = dr.sub('*',strloc)
                    for b in blacklist:
                        if b in str1:
                            str1 = str1.replace(b,'+')
                    elem.value = str1
                    #date/pid finder
                    x = re.findall('[0-9]{6,15}', str1)
                    if len(x) != 0:
                        #creating df to look into results
                        patient.append(df.iloc[i][1])
                        study.append(df.iloc[i][2])
                        series.append(df.iloc[i][3])
                        SOP.append(df.iloc[i][4])
                        element.append(str(elem))
                        tag.append(elem.name)
                        vr.append(elem.VR)
                        value.append(str(elem.value))
                        
                        for n in x:
                            str1 = elem.value
                            #date format change
                            if len(n) == 8:
                                yr = n[:4]
                                m = n[4:6]
                                d = n[6:]
                                if 1895<=int(yr)<=2023 and 0<int(m)<=12 and 0<int(d)<=31:
                                    if elem.VR == 'IS':
                                        #remove dates in IS VRs
                                        str2 = str1.replace(n,'')
                                        elem.value = str2
                                    else:
                                        #reformat dates in all other VRs
                                        str3 = yr+'-'+m+'-'+d
                                        str4 = str1.replace(n,str3)
                                        elem.value = str4
                                else:
                                    #replace non-dates len of 8
                                    str2 = str1.replace(n,'')
                                    elem.value = str2
                            else:
                                #replace non-len of 8
                                str2 = str1.replace(n,'')
                                elem.value = str2
                            #add new val to df
                        newvalue.append(str(elem.value))
                                
                except:
                    print(elem)
                    

,patient,study,series,SOP,tag,vr,element,value
0,1326414678,2.2.379.0.1.5971236.3.509.2117853776506399936,2.2.379.0.1.5971236.3.509.6412913489296797476,2.2.379.0.1.5971236.3.509.1118646326684973163,Accession Number,SH,"(0008, 0050) Accession Number ...",20170304E
1,1507439060,2.2.874.0.0.8400978.5.486.2510732133849908984,2.2.874.0.0.8400978.5.486.1747132007124743752,2.2.874.0.0.8400978.5.486.2019300706932341803,Accession Number,SH,"(0008, 0050) Accession Number ...",20170726E
2,1814567196,1.5.566.0.3.5921028.1.523.6121491292105488144,1.5.566.0.3.5921028.1.523.5821238071079767946,1.5.566.0.3.5921028.1.523.6321575997177780656,Accession Number,SH,"(0008, 0050) Accession Number ...",20180805E
3,4953814140,1.3.241.0.0.3328322.7.935.2685937606730192613,1.3.241.0.0.3328322.7.935.9222594282780660977,1.3.241.0.0.3328322.7.935.1641820698690379306,Accession Number,SH,"(0008, 0050) Accession Number ...",20171114E
4,4953814140,1.3.241.0.0.3328322.7.935.2685937606730192613,1.3.241.0.0.3328322.7.935.9222594282780660977,1.3.241.0.0.3328322.7.935.3386585305478125613,Accession Number,SH,"(0008, 0050) Accession Number ...",20171114E
...,...,...,...,...,...,...,...,...
348,9473050187,2.5.444.1.3.9432617.9.170.1605644608138802224,2.5.444.1.3.9432617.9.170.1185165576631833947,2.5.444.1.3.9432617.9.170.1220587225385277156,Accession Number,SH,"(0008, 0050) Accession Number ...",20140722E
349,9473050187,2.5.444.1.3.9432617.9.170.1605644608138802224,2.5.444.1.3.9432617.9.170.1185165576631833947,2.5.444.1.3.9432617.9.170.3053293679157621944,Accession Number,SH,"(0008, 0050) Accession Number ...",20140722E
350,9473050187,2.5.444.1.3.9432617.9.170.1605644608138802224,2.5.444.1.3.9432617.9.170.1185165576631833947,2.5.444.1.3.9432617.9.170.2941125927500826196,Accession Number,SH,"(0008, 0050) Accession Number ...",20140722E
351,9473050187,2.5.444.1.3.9432617.9.170.1605644608138802224,2.5.444.1.3.9432617.9.170.1185165576631833947,2.5.444.1.3.9432617.9.170.3114818742644719069,Accession Number,SH,"(0008, 0050) Accession Number ...",20140722E


In [ ]:
bucket = client.get_bucket('midi-import')
for i in dicomlist:
    filepath = 'preprocessed/ds1_1-23preprocessed5-19/'+filelist_in[1+i]
    blob = bucket.blob(filepath)
    df.iloc[i][0].save_as('test2.dcm')
    blob.upload_from_filename('test2.dcm')
    

In [35]:
df.iloc[30][0]['SeriesNumber']

(0020, 0011) Series Number                       IS: ''

In [ ]:
#manual code final 
file = 0
vrs = ['LO', 'SH', 'LT', 'ST', 'UT','IS']
str1 = ''
x = []
x2 = []

patient = []
study = []
SOP = []
series = []
element = []
tag = []
value = []
vr = []

for i in range(len(df)):
    file = df.iloc[i][0]
    for elem in file.iterall():
        if elem.VR in vrs:
            if str(elem.value).isalpha() == False:
                str1 = elem.value
                try:
                    x = re.findall('[0-9]+', str1)
                    x2 = list(filter(filterLength, x))
                    if len(x2) != 0:
                        patient.append(df.iloc[i][1])
                        study.append(df.iloc[i][2])
                        series.append(df.iloc[i][3])
                        SOP.append(df.iloc[i][4])
                        element.append(str(elem))
                        tag.append(elem.name)
                        vr.append(elem.VR)
                        value.append(elem.value)
                except:
                    print(elem)
                                      
                        
    

In [ ]:
hkkj  ll4kk

In [6]:
file = 0
vrs = ['LO', 'SH', 'LT', 'ST', 'UT','IS', 'UL', 'US']
str1 = ''
x = []
x2 = []

patient = []
study = []
SOP = []
series = []
element = []
tag = []
value = []
vr = []

for i in range(len(df)):
    file = df.iloc[i][0]
    for elem in file.iterall():
        if elem.VR in vrs:
            if str(elem.value).isalpha() == False:
                str1 = str(elem.value)
                try:
                    x = re.findall('[0-9]+', str1)
                    x2 = list(filter(filterLength, x))
                    if len(x2) != 0:
                        patient.append(df.iloc[i][1])
                        study.append(df.iloc[i][2])
                        series.append(df.iloc[i][3])
                        SOP.append(df.iloc[i][4])
                        element.append(str(elem))
                        tag.append(elem.name)
                        vr.append(elem.VR)
                        value.append(elem.value)
                except:
                    print(elem)
                                      
                        
    

/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.1.335.1.3.0612302.1.183.1432606318663715083'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '2.3.439.1.0.0712194.8.594.1419903147336527125'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '2.3.439.1.0.0712194.8.594.2801993338883413721'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: The value length (72) exceeds

In [40]:
str1

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [5]:
file = 0
vrs = ['LO', 'SH', 'LT', 'ST', 'UT', 'IS', 'UL', 'US']
str1 = ''
x = []
x2 = []

patient = []
study = []
SOP = []
series = []
element = []
tag = []
value = []
vr = []

for i in range(len(df)):
    file = df.iloc[i][0]
    for elem in file.iterall():
        if str(elem.value).isalpha() == False:
            str1 = str(elem.value)
            try:
                x = re.findall('[0-9]+', str1)
                x2 = list(filter(filterLength, x))
                if len(x2) != 0:
                    patient.append(df.iloc[i][1])
                    study.append(df.iloc[i][2])
                    series.append(df.iloc[i][3])
                    SOP.append(df.iloc[i][4])
                    element.append(str(elem))
                    tag.append(elem.name)
                    vr.append(elem.VR)
                    value.append(elem.value)
            except:
                print(elem)
                                      
                        
    

/opt/conda/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '2.5.428.0.3.5852292.6.070.1246301751979395525'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)


KeyboardInterrupt: 

In [1]:
file = 0
vrs = ['LO', 'SH', 'LT', 'ST', 'UT']
str1 = ''
x = []
x2 = []

patient = []
study = []
SOP = []
series = []
element = []
tag = []
value = []
vr = []

for i in range(len(df)):
    file = df.iloc[i][0]
    for elem in file.iterall():
        if str(elem.value).isalpha() == False:
            str1 = str(elem.value)
            try:
                x = re.findall('[0-9]+', str1)
                x2 = list(filter(filterLength, x))
                #if len(x2) != 0:
                    #patient.append(df.iloc[i][1])
                    #study.append(df.iloc[i][2])
                    #series.append(df.iloc[i][3])
                    #SOP.append(df.iloc[i][4])
                    #element.append(str(elem))
                    #tag.append(elem.name)
                    #vr.append(elem.VR)
                    #value.append(elem.value)
            except:
                print(elem)
                                      
                        
    

NameError: name 'df' is not defined

In [28]:
d = {'patient':patient, 'study':study, 'series':series, 'SOP':SOP, 'tag':tag, 'vr':vr, 'element':element, 'value':value}
flagged = pd.DataFrame(d)

In [31]:
flagged.groupby(['tag'])['tag'].count()

tag
Accession Number    353
Name: tag, dtype: int64

In [8]:
flagged2 = flagged[flagged['tag']!='Code Value']
flagged2 = flagged2[flagged2['tag']!='Patient ID']
flagged2 = flagged2[flagged2['tag']!='Study ID']
flagged2 = flagged2[flagged2['tag']!='Clinical Trial Subject ID']
flagged2 = flagged2[flagged2['tag']!='Accession Number']

In [9]:
flagged2['tag'].unique()

array(['Additional Patient History', 'Patient Comments',
       'Actual Frame Duration', 'Series Description', 'Study Comments',
       'Series Number', 'Scheduled Procedure Step Description',
       'Study Description', 'Private Creator', 'Protocol Name',
       'Requested Procedure Comments', 'Requested Procedure Description',
       'Imaging Service Request Comments',
       'Attenuation Correction Method', 'Acquisition Comments',
       'Private tag data', 'Image Comments', 'Text Value',
       'Medical Alerts'], dtype=object)

In [10]:
len(flagged)

7161

In [11]:
unique_flagged = flagged2.drop_duplicates(subset=['study','element'])

In [12]:
len(unique_flagged)

89

In [25]:
unique_flagged.to_csv('allflaggedtest.csv')

In [102]:
unique_flagged[unique_flagged['tag']=='Image Comments']

,study,series,SOP,tag,vr,element
1304,1.3.241.0.0.3328322.7.935.2685937606730192613,1.3.241.0.0.3328322.7.935.9222594282780660977,1.3.241.0.0.3328322.7.935.2137823494280056407,Image Comments,LT,"(0020, 4000) Image Comments ..."
2114,1.3.241.0.0.3328322.7.935.2685937606730192613,1.3.241.0.0.3328322.7.935.9222594282780660977,1.3.241.0.0.3328322.7.935.3037455751552642927,Image Comments,LT,"(0020, 4000) Image Comments ..."
3874,2.3.439.1.0.0712194.8.594.3353626527435595311,2.3.439.1.0.0712194.8.594.2584876857362372891,2.3.439.1.0.0712194.8.594.2118017278604801017,Image Comments,LT,"(0020, 4000) Image Comments ..."
4050,2.3.439.1.0.0712194.8.594.3353626527435595311,2.3.439.1.0.0712194.8.594.2584876857362372891,2.3.439.1.0.0712194.8.594.3217557278435333498,Image Comments,LT,"(0020, 4000) Image Comments ..."
4333,2.5.444.1.3.9432617.9.170.1605644608138802224,2.5.444.1.3.9432617.9.170.1185165576631833947,2.5.444.1.3.9432617.9.170.2630043865913544067,Image Comments,LT,"(0020, 4000) Image Comments ..."
4416,2.5.760.0.0.0071116.8.497.2211300014030572904,2.5.760.0.0.0071116.8.497.6426282347040498683,2.5.760.0.0.0071116.8.497.1302127158402175177,Image Comments,LT,"(0020, 4000) Image Comments ..."
4479,2.5.760.0.0.0071116.8.497.2211300014030572904,2.5.760.0.0.0071116.8.497.6426282347040498683,2.5.760.0.0.0071116.8.497.2112746267588715374,Image Comments,LT,"(0020, 4000) Image Comments ..."
4606,2.5.760.0.0.0071116.8.497.2211300014030572904,2.5.760.0.0.0071116.8.497.6426282347040498683,2.5.760.0.0.0071116.8.497.6996487001063066160,Image Comments,LT,"(0020, 4000) Image Comments ..."


In [122]:
unique_flagged['SOP'][1304]

'1.3.241.0.0.3328322.7.935.2137823494280056407'

In [123]:
df[df['SOPID']=='1.3.241.0.0.3328322.7.935.2137823494280056407']

,dicom,patientID,studyID,seriesID,SOPID
176,"[(0008, 0005) Specific Character Set ...",PLACEHOLDER,1.3.241.0.0.3328322.7.935.2685937606730192613,1.3.241.0.0.3328322.7.935.9222594282780660977,1.3.241.0.0.3328322.7.935.2137823494280056407


In [125]:
df.iloc[176][0]['ImageComments'].value

'Ordered for 4953814140 by MC'